In [45]:
import torch
import numpy as np
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import pandas as pd

## question 3

In [46]:
def kernelFunc(x1, x2, kernel = 'linear', sigma = 1.0, p = 2):
    if kernel == 'linear':
        return np.dot(x1, x2)
    elif kernel == 'gaussian':
        return np.exp(-np.linalg.norm(x1 - x2)**2/(2*(sigma**2)))
    elif kernel == 'exponent':
        return np.exp(np.dot(x1, x2))
    elif kernel == 'polynomial':
        return (1 + np.dot(x1, x2))**p

In [47]:
def SVM(x, y, kernel = 'linear', sigma = 1.0, p = 2):
    n, d = x.shape
    y = y.reshape(-1, 1)
    temp = np.zeros((n, n), dtype = float)
    for i in range(n):
        for j in range(n):
            temp[i, j] = kernelFunc(x[i], x[j], kernel, sigma, p)
    P = cvxopt_matrix(np.outer(y,y)*temp)
    q = cvxopt_matrix(-np.ones((n, 1), dtype = float))
    G = cvxopt_matrix(-np.eye(n))
    h = cvxopt_matrix(np.zeros(n, dtype = float))
    A = cvxopt_matrix(y.T)
    b = cvxopt_matrix(np.zeros(1, dtype = float))

    cvxopt_solvers.options['show_progress'] = False
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10

    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    beta = ((y*alphas).T@x).reshape(-1, 1)
    s = (alphas > 1e-10).flatten()
    #first element of beta0 array is used in prediction
    beta0 = (y[s] - np.dot(x[s], beta))[0]
    #mean of beta0 array is used in prediction
    # beta0 = np.mean((y[s] - np.dot(x[s], beta)))
    return alphas, beta, beta0

def predictSVM(x, alphas, beta, beta0, kernel = 'linear', sigma = 1.0, p = 2):
    # result = []
    # for i in range(len(x)):
    #     print("kernel value : ", kernelFunc(x[i], beta, kernel, sigma, p))
    #     result.append(np.sign([(kernelFunc(x[i], beta, kernel, sigma, p) + beta0)]))
    # return np.array(result)
    return np.sign(np.dot(x, beta) + beta0)

## question 4

In [48]:
x_train = pd.read_csv('Xsvm.csv', header=None).values
y_train = pd.read_csv('ysvm.csv', header=None).values
x_test = np.array([[1.0, 0.4], [0.9, 0.9], [1.4, 1.5], [0.01, 0.005]])
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (500, 2)
y_train shape:  (500, 1)
x_test shape:  (4, 2)


below are the results from linear svm and other kernel svm for training on csv data files when first element of β0 (offset) array is used as β0 for calculation of predictions. When mean of β0 (offset) array while training is used in prediction then the results changes for all types of kernels .


In [49]:
alphas, beta, beta0 = SVM(x_train, y_train)
print(predictSVM(x_test, alphas, beta, beta0))

[[1.]
 [1.]
 [1.]
 [1.]]


In [50]:
alphas, beta, beta0 = SVM(x_train, y_train, kernel = 'gaussian', sigma = 0.001)
print(predictSVM(x_test, alphas, beta, beta0))

[[ 1.]
 [ 1.]
 [ 1.]
 [-1.]]


In [51]:
alphas, beta, beta0 = SVM(x_train, y_train, kernel = 'exponent')
print(predictSVM(x_test, alphas, beta, beta0))

[[1.]
 [1.]
 [1.]
 [1.]]


In [52]:
alphas, beta, beta0 = SVM(x_train, y_train, kernel = 'polynomial', p = 2)
print(predictSVM(x_test, alphas, beta, beta0))

[[ 1.]
 [ 1.]
 [ 1.]
 [-1.]]


## question 5

below are the results from linear svm and other kernel svm for training on XOR dataset when first element of β0 (offset) array is used as β0 for calculation of predictions. When mean of β0 (offset) array while training is used in prediction then the results changes for all types of kernels giving a prediction of all 0s in linear, gaussian and exponent kernel and an alternate prediction of 1 and -1 that's [1, -1, 1, -1] in polynomial kernel.

In [40]:
x_train = np.array([[-1, -1], [-1, 1], [1, -1], [1, 1]])*1.
y_train = np.array([-1, 1, 1, -1])*1.

In [41]:
alphas, beta, beta0 = SVM(x_train, y_train)
print(predictSVM(x_train, alphas, beta, beta0))

[[-1.]
 [-1.]
 [-1.]
 [-1.]]


In [42]:
alphas, beta, beta0 = SVM(x_train, y_train, kernel = 'gaussian', sigma = 0.01)
print(predictSVM(x_train, alphas, beta, beta0, kernel = 'gaussian', sigma = 0.01))

[[-1.]
 [-1.]
 [-1.]
 [-1.]]


In [43]:
alphas, beta, beta0 = SVM(x_train, y_train, kernel = 'exponent')
print(predictSVM(x_train, alphas, beta, beta0, kernel = 'exponent'))

[[-1.]
 [-1.]
 [-1.]
 [-1.]]


In [44]:
alphas, beta, beta0 = SVM(x_train, y_train, kernel = 'polynomial', p=2)
print(predictSVM(x_train, alphas, beta, beta0, kernel = 'polynomial', p=2))

[[-1.]
 [-1.]
 [-1.]
 [-1.]]
